In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from tqdm import tqdm
from random import shuffle
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dense,Dropout,Flatten
from keras import optimizers
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import keras
from classification_models import ResNet18
import sys
import os
from time import time
from keras.models import load_model
from keras.models import Sequential
"""
Adapted from keras example cifar10_cnn.py
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py
"""
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping, TensorBoard, LearningRateScheduler
from keras import backend as K
import numpy as np
from keras.layers import AveragePooling2D
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    Dropout
)
from keras.layers import Dense
from keras.models import Model
from keras import optimizers


In [3]:
TRAIN_DIR='/home/rahulchakwate/My_tensorflow/DataSets/cats_and_dogs_2/train'
IMG_SIZE=224
LR=1e-3
MODEL_NAME='cat-dog-convnet'

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

def create_label(image_name):
    word_label=image_name.split('.')[-3]
    if word_label=='cat':
        return np.array([1,0])
    elif word_label=='dog':
        return np.array([0,1])


In [4]:
def create_train_data():
    training_data=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path=os.path.join(TRAIN_DIR,img)
        img_data=cv2.imread(path,cv2.IMREAD_COLOR)
        img_data=cv2.resize(img_data,(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img_data),create_label(img)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data


In [ ]:
train_data=create_train_data()

 22%|██▏       | 5489/25000 [01:30<05:20, 60.90it/s]

In [ ]:
train=train_data
X_train=np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
y_train=np.array([i[1] for i in train])
X_train=
print(X_train.shape)
print(y_train.shape)

In [75]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 20,40 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-5
    if epoch > 20:
        lr = lr*0.1
    return lr

#lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

In [17]:
base_model = ResNet18(input_shape=(50,50,3), weights='imagenet', include_top=False)

In [19]:
pool2 = AveragePooling2D(pool_size=(4,4),
                                 strides=(1, 1))(base_model.output)
base_model1=Model(inputs=base_model.input, outputs=pool2)

In [20]:
base_model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 50, 50, 3)    0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 50, 50, 3)    9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 56, 56, 3)    0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 25, 25, 64)   9408        zero_padding2d_19[0][0]          
__________________________________________________________________________________________________
bn0 (Batch

In [58]:
bottleneck_features_train=base_model1.predict(X_train[:22000], verbose=1)

22000/22000 [==============================] - 298s 14ms/step


In [59]:
bottleneck_features_val=base_model1.predict(X_train[22000:], verbose=1)

3000/3000 [==============================] - 40s 13ms/step


In [60]:
print(bottleneck_features_train.shape)
print(bottleneck_features_val.shape)

(22000, 1, 1, 512)
(3000, 1, 1, 512)


In [76]:
nb_classes=2
train_data=bottleneck_features_train
validation_data=bottleneck_features_val
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax',kernel_initializer='random_normal'))

tensorboard = TensorBoard(log_dir="log2/{}".format(time()))

adam=optimizers.Adam(lr=1e-5)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [77]:
model.fit(train_data, y_train[:22000],
              nb_epoch=200,
              validation_data=(validation_data, y_train[22000:]),
              shuffle=True,
              callbacks=[tensorboard, lr_scheduler])

/home/rahulchakwate/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 22000 samples, validate on 3000 samples
Epoch 1/200

Epoch 00001: LearningRateScheduler setting learning rate to 1e-05.
22000/22000 [==============================] - 4s 167us/step - loss: 0.8008 - acc: 0.5790 - val_loss: 0.5863 - val_acc: 0.6947
Epoch 2/200

Epoch 00002: LearningRateScheduler setting learning rate to 1e-05.
22000/22000 [==============================] - 3s 132us/step - loss: 0.6493 - acc: 0.6593 - val_loss: 0.5353 - val_acc: 0.7360
Epoch 3/200

Epoch 00003: LearningRateScheduler setting learning rate to 1e-05.
22000/22000 [==============================] - 2s 107us/step - loss: 0.5921 - acc: 0.6996 - val_loss: 0.5121 - val_acc: 0.7587
Epoch 4/200

Epoch 00004: LearningRateScheduler setting learning rate to 1e-05.
22000/22000 [==============================] - 2s 105us/step - loss: 0.5624 - acc: 0.7150 - val_loss: 0.4983 - val_acc: 0.7693
Epoch 5/200

Epoch 00005: LearningRateScheduler setting learning rate to 1e-05.
22000/22000 [==============================

KeyboardInterrupt: 

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
     os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
model2=load_model(model_path)

In [ ]:
history2=model2.fit(x=X_train,y=y_train,batch_size=4,epochs=10,verbose=1,validation_split=0.2)

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
     os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model2.save(model_path)
print('Saved trained model at %s ' % model_path)

In [ ]:
model3=load_model(model_path)

In [ ]:
adam=optimizers.Adam(lr=0.00001)
model3.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history3=model3.fit(x=X_train,y=y_train,batch_size=4,epochs=5,verbose=1,validation_split=0.2)

In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
keras.callbacks.TensorBoard(log_dir='/tmp/log')